# Simple CCZ4 Finite Volume code
 
A solver of the SWE equations that uses the original ExaHyPE FORTRAN kernels.

In [1]:
import os
import peano4
import exahype2

DaStGen 2 (C) www.peano-framework.org
Peano 4 (C) www.peano-framework.org
ExaHyPE 2 (C) www.peano-framework.org


In [2]:
output_files = [ f for f in os.listdir(".") if f.endswith(".peano-patch-file") or f.endswith(".vtu") or f.startswith("output")]
for f in output_files:
  os.remove(f)

The first thing to do in any ExaHyPE 2 application is to create an ExaHyPE project. We have to tell it exactly what namespace we want to use and we have to give it a name. There are a few more options that we omit here, i.e. we use the defaults.

In [3]:
project = exahype2.Project( ["examples", "exahype2", "ccz4"], "ccz4" )

## Configuring the solver

Our first step is to use a Finite Volumes solver. I write down all the unknowns first. I'll use this dictionary later to access variables symbolically. For the time being, I just define this map and then count the number of unknowns. This total count is something I really need to configure my solver.

In [4]:
unknowns = {
  "G":6,
  "K":6,
  "theta":1,
  "Z":3,
  "lapse":1,
  "shift":3,
  "b":3,
  "dLapse":3,
  "dxShift":3,
  "dyShift":3,
  "dzShift":3,
  "dxG":6,
  "dyG":6,
  "dzG":6,
  "traceK":1,
  "phi":1,
  "P":3,
  "K0":1,
  "domain":1,
  "pos":3
}

number_of_unknowns = 0
for i in unknowns:
  number_of_unknowns += unknowns[i]
print( "number of unknowns=", number_of_unknowns )

number of unknowns= 63


The solver itself is a simple Finite Volume solver with fixed time stepping. I use the enclave solver variant which is a solver nuance that is particuarly good on large shared memory architectures. Our PDE is expressed solely in a nonconservative formulation. I therefore can switch the flux term off.

In [5]:
patch_size          = 11
auxiliary_variables = 0
time_step_size      = 0.0001
max_h               = 90
min_h               = 90

my_solver = exahype2.solvers.fv.GenericRusanovFixedTimeStepSizeWithEnclaves(
  "CCZ4", patch_size, number_of_unknowns, auxiliary_variables, min_h, max_h, 0.0001
)


project.add_solver(my_solver)


I next inject homogeneous Neumann boundary conditions:

In [6]:
import exahype2.sympy

pde = exahype2.sympy.PDE(unknowns=number_of_unknowns,auxiliary_variables=0,dimensions=3)

my_solver.set_implementation(
  boundary_conditions=pde.implementation_of_homogeneous_Neumann_BC(),
  flux=exahype2.solvers.fv.PDETerms.None_Implementation,
  ncp=exahype2.solvers.fv.PDETerms.User_Defined_Implementation
)   


## Configure the global domain

We next configure our global domain, i.e. we specify the dimensions of the computational domain $\Omega $, we specify how long the simulation shall run and how often we want it to dump its data. The dumps will later be used to produce screenshots of the data (or to extract all kinds of properties).

In [7]:
build_mode = peano4.output.CompileMode.Asserts
#build_mode = peano4.output.CompileMode.Release

dimensions = 3

end_time = 1
    
project.set_global_simulation_parameters(
  dimensions,               # dimensions
  [-150.0, -150.0, -150.0],  [300.0, 300.0, 300.0],
  end_time,                 # end time
  0.0, time_step_size*10    # snapshots
)

## Generate a Peano 4 project

This is exactly the same as for Euler: We ask the ExaHyPE 2 front-end to give us a Peano project.

In [8]:
project.set_Peano4_installation("../../..", build_mode)

peano4_project = project.generate_Peano4_project()

parse configure outcome ../../../src/Makefile to extract compile settings
used C++ compiler is icpc
used Fortran compiler is ifort


A "big" difference to Euler is that we now have some external Fortran files. Usually, we take the default settings when we invoke the Fortran compiler. Default is the stuff given to us by Peano's configure. There's however the opportunity to add further flags manually. I use this here as the astro code requies some bespoke defines. Also, I need the -cpp flag. Otherwise the preprocessor macros are not taken into account properly and my Intel compiler terminates.

In [9]:
peano4_project.output.makefile.add_Fortran_flag( "-DCCZ4EINSTEIN -DGLMROT -DDim3" )
peano4_project.output.makefile.add_Fortran_flag( "-r8 -cpp -auto" )
peano4_project.output.makefile.add_Fortran_files( 
  ["MainVariables.f90", "PDE.f90", "ICTools.f90", 
   "ADMConstraints.f90", "InitialData.f90", "Metric.f90", "C2P-FOCCZ4.f90"] 
)

We finally generate the C++ code:

In [10]:
peano4_project.generate( throw_away_data_after_generation=False )

generate all code ...
some artefacts have already been added to repository ... assume this is intentional (by higher abstraction layer, e.g.)
generated ccz4-main.cpp
write ./Makefile
write ././AbstractCCZ4.h (generated from template /home/tobias/git/Peano/python/exahype2/solvers/fv/GenericRusanovFixedTimeStepSizeWithEnclavesAbstract.template.h)
write ././AbstractCCZ4.cpp (generated from template /home/tobias/git/Peano/python/exahype2/solvers/fv/GenericRusanovFixedTimeStepSizeWithEnclavesAbstract.template.cpp)
write ./observers/SolverRepository.h (generated from template /home/tobias/git/Peano/python/exahype2/ProjectSolverRepository.template.h)
write ./observers/SolverRepository.cpp (generated from template /home/tobias/git/Peano/python/exahype2/ProjectSolverRepository.template.cpp)
write ./celldata/CCZ4Q.h (generated from template /home/tobias/git/Peano/python/peano4/datamodel/PatchToDoubleArray.template.h)
write ./celldata/CCZ4Q.cpp (generated from template /home/tobias/git/Peano/pyth

The build below should fail when you first call it in an empty directory. After all, we have promised to deliver an ncp implementation, but there is none.

In [11]:

parallel_builds = 8   # I don't use a massively parallel build here as my laptop otherwise becomes too hot.
                      # Without any arguments, the build process will grab all of your cores.
peano4_project.build( make_clean_first = True, number_of_parallel_builds = parallel_builds )

clean up project ...
clean complete
start to compile with concurrency level of 8 ...
icpc -std=c++0x -g -O2 -DPeanoDebug=2 -I/opt/intel/tbb/include -DTBB_USE_ASSERT -DTBB_USE_THREADING_TOOLS -I/opt/intel/itac/2020.0.015/intel64/include -fopenmp  -DDimensions=3 -I. -I../../../src -c -o AbstractCCZ4.o AbstractCCZ4.cpp
icpc -std=c++0x -g -O2 -DPeanoDebug=2 -I/opt/intel/tbb/include -DTBB_USE_ASSERT -DTBB_USE_THREADING_TOOLS -I/opt/intel/itac/2020.0.015/intel64/include -fopenmp  -DDimensions=3 -I. -I../../../src -c -o CCZ4.o CCZ4.cpp
icpc -std=c++0x -g -O2 -DPeanoDebug=2 -I/opt/intel/tbb/include -DTBB_USE_ASSERT -DTBB_USE_THREADING_TOOLS -I/opt/intel/itac/2020.0.015/intel64/include -fopenmp  -DDimensions=3 -I. -I../../../src -c -o observers/SolverRepository.o observers/SolverRepository.cpp
icpc -std=c++0x -g -O2 -DPeanoDebug=2 -I/opt/intel/tbb/include -DTBB_USE_ASSERT -DTBB_USE_THREADING_TOOLS -I/opt/intel/itac/2020.0.015/intel64/include -fopenmp  -DDimensions=3 -I. -I../../../src -c -o cel

## Run the code and postprocess results

Once we have the Peano 4 project set up and built, we obtain an executable which is standalone. It comprises all the parameters we have set above. So we can either invoke this file on the command line or we can call it through the notebook. Depending on your setup you might want to call the code with a prefix. If you configured Peano with MPI, e.g., you might have to use the mpirun prefix. 

Please note that we pipe the terminal output into a file. This way, we can postprocess the data in the next step.

In [12]:
peano4_project.run( [], prefix = ["mpirun", "-n", "1"], pipefile = "output.txt", rebuild_if_required=False )

run application ...


KeyboardInterrupt: 

We first grab and postprocess the data from the output file. This is ExaHyPE 2-specific (though application-generic), so we rely on ExaHyPE's postprocessing routines.



In [ ]:
import exahype2.postprocessing

performance_data = exahype2.postprocessing.PerformanceData( "output.txt", verbose=True )

In this worksheet, I'd like to have interactive matplotlib plots: You might need the statement

<pre>
%matplotlib widget
</pre>

but on some of my local notebook servers, this causes a crash. So you have to test this yourself.

In [ ]:
exahype2.postprocessing.plot_pie_chart_over_simulation_phases(performance_data)

In [ ]:
exahype2.postprocessing.plot_time_per_time_step(performance_data)

## Inspecting output files via Paraview

We first do an ls on our directory and search for the root .peano-patch-file. I then convert this file manually into vtu which I can load into Paraview. This works if Peano is configured with vtk support. If you don't have vtk support in there, then you might want to load the data directly into Paraview through Peano's Paraview plug-in.

In [ ]:
!ls